# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [16]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [4]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
    print("Connection Established !!")
except Exception as e:
    print(f"Connection Failed !! Error : {e}")

Connection Established !!


#### Create Keyspace

In [5]:
# TO-DO: Create a Keyspace
query_keyspace = """CREATE KEYSPACE IF NOT EXISTS sparkify with REPLICATION = 
                    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
                """

# Creating Keyspace
try:
    session.execute(query_keyspace)
except Exception as e:
    print(f"Failed to create keyspace!! Error : {e}")

#### Set Keyspace

In [6]:
# TO-DO: Set KEYSPACE to the keyspace specified above
session.set_keyspace('sparkify')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
### ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
## QUERY #1
### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

#### For this question we will need to make a query that returns the following:

#### - The artist name
#### - Song name
####  - Song Lenght

#### These results will be obtained from the table "session_songs" filtered by "sessionId" and "itemInSession"

#### Our CQL query will be:

#### SELECT artist, song_title, song_length FROM session_songs WHERE sessionId = 338 AND itemInSession = 4

#### Additionally:
#### - The PRIMARY KEY is "(sessionId, itemInSession)".
#### - The columns of the table are sessionId, itemInSession, artist, song_title, song_length
#### - The column sessionId is the partition key and itemInSession is the clustering column.

#### CREATE TABLE session_songs

In [22]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
create_table_query1 = ("""
    CREATE TABLE IF NOT EXISTS session_songs
    (sessionId int, itemInSession int, artist text, song_title text, song_length float,
    PRIMARY KEY(sessionId, itemInSession))
    """)
try: 
    session.execute(query1)
    print("The table is Created in the project!!!")
except Exception as e:
    print(f"Table creation failed!! Error : {e}")

The table is Created in the project!!!


#### INSERT THE DATA IN THE TABLE session_songs

In [23]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO session_songs (sessionId, itemInSession, artist, song_title, song_length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        artist_name, user_name, gender, itemInSession, user_last_name, length, level, location, sessionId, song, userId = line
        session.execute(query, (int(sessionId), int(itemInSession), artist_name, song, float(length)))

#### Do a SELECT to verify that the data have been inserted into each table session_songs

In [24]:
# SELECT statement to verify the data was entered into the table
select_query1 = "SELECT artist, song_title, song_length FROM session_songs WHERE sessionId = 338 AND itemInSession = 4"
try:
    rows = session.execute(select_query1)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)

Row(artist='Faithless', song_title='Music Matters (Mark Knight Dub)', song_length=495.30731201171875)


### ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
## QUERY #2

### Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

#### For this question we will need to make a query that returns the following:

#### - The artist
#### - The song
#### - The firstName
#### - The lastname

#### These results will be obtained from the table "user_session" filtered by "sessionId" and "userId"

#### Our CQL query will be:

#### SELECT artist, song, firstName, lastName FROM  user_session where sessionId = 182 and userId = 10

#### Additionally:
#### - The PRIMARY KEY is "((sessionId, userId), itemInSession)".
#### - In the table it will be used "WITH CLUSTERING ORDER BY (itemInSession ASC)".
#### - The columns of the table are sessionId, userId, artist, song, firstName, lastName, itemInSession
#### - The columns (sessionId, userId) is the partition key and itemInSession is the clustering column.

#### CREATE TABLE user_session

In [25]:
create_table_query2 = """CREATE TABLE IF NOT EXISTS user_session (sessionId int, userId int, artist text, song text, firstName text, lastName text, itemInSession int, PRIMARY KEY ((sessionId, userId), itemInSession)) WITH CLUSTERING ORDER BY (itemInSession ASC) """

try: 
    session.execute(create_table_query2)
    print("The table is Created in the project!!!")
except Exception as e:
    print(f"Table creation failed!! Error : {e}")

The table is Created in the project!!!


#### INSERT THE DATA IN THE TABLE user_session

In [26]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_session (sessionId, userId, artist, song, firstName, lastName, itemInSession) "
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s) "
        session.execute(query, (int(line[8]), int(line[10]), line[0], line[9], line[1], line[4], int(line[3])  ) )

#### Do a SELECT to verify that the data have been inserted into each table user_session

In [27]:
select_query2 = "SELECT artist, song, firstName, lastName FROM  user_session where sessionId = 182 and userId = 10"
try:
    rows = session.execute(select_query2)
except Exception as e:
    print(e)

for row in rows:
    print(row)

Row(artist='Down To The Bone', song="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')
Row(artist='Three Drives', song='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', firstname='Sylvie', lastname='Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')


### ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
## QUERY #3

### Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

#### For this question we will need to make a query that returns the following:

#### - The song
#### - The firstName
#### - The lastname

#### These results will be obtained from the table "user_song" filtered by "song"

#### Our CQL query will be:

#### SELECT song, firstName, lastName FROM user_song where song = 'All Hands Against His Own'

#### Additionally:
#### - The PRIMARY KEY is "((song), userId))".
#### - The columns of the table are song, userId, firstName, lastName
#### - The column (song) is the partition key and userId is the clustering column.

#### CREATE TABLE user_song

In [28]:
create_table_query3 = """CREATE TABLE IF NOT EXISTS user_song (song text, userId int, firstName text, lastName text, PRIMARY KEY ((song), userId))"""

try: 
    session.execute(create_table_query3)
    print("The table is Created in the project!!!")
except Exception as e:
    print(f"Table creation failed!! Error : {e}")

The table is Created in the project!!!


#### INSERT THE DATA IN THE TABLE user_song

In [29]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_song (song, userId, firstName, lastName) "
        query = query + " VALUES (%s, %s, %s, %s) "
        session.execute(query, (  line[9], int(line[10]), line[1], line[4] )  )

#### SELECT statement to verify the data was entered into the table user_song

In [30]:
select_query3 = "SELECT song, firstName, lastName FROM user_song where song = 'All Hands Against His Own'"
try:
    rows = session.execute(select_query3)
except Exception as e:
    print(e)

for row in rows:
    print(row)

Row(song='All Hands Against His Own', firstname='Jacqueline', lastname='Lynch')
Row(song='All Hands Against His Own', firstname='Tegan', lastname='Levine')
Row(song='All Hands Against His Own', firstname='Sara', lastname='Johnson')


### Drop the tables before closing out the sessions

In [31]:
session.execute("DROP TABLE IF EXISTS sparkify.session_item")
session.execute("DROP TABLE IF EXISTS sparkify.user_session")
session.execute("DROP TABLE IF EXISTS sparkify.user_song")

### Close the session and cluster connection¶

In [32]:
session.shutdown()
cluster.shutdown()